## Init

## mini agent calculators

In [324]:
import numpy as np
import tensorflow as tf

data = np.array([10, 15, 13, 19, 14, 18, 17, 11])
EQUATION_ARR = [np.asarray([0]*(5))]*4 # Система нормальних рівнянь
AGENTS = []
prediction_type = 1

class Gaus_line_equation:
    def __init__(self, y, F1, F2, coef3):
        self.y = y
        self.coefs_a = [1, F1, F2, coef3]
        self.array = []
        for i in range(len(self.coefs_a)):
            temp_arr = [self.y]+self.coefs_a
            x = np.asarray(temp_arr)*self.coefs_a[i]
            self.array.append(x)


def get_coef3(i):
    if prediction_type == 2:
        coef = pow(data[i-2],2)* data[i-1]
    elif prediction_type == 3:
        coef = data[i-2]* pow(data[i-1],2)
    else:
        coef = data[i-2]* data[i-1]
    return coef

# функція похибки
def mean_square_deviation(real, predicted):
    s = 0
    for i in range(len(real)):
        s += pow(real[i] - predicted[i], 2)
    return pow(s/len(real), 0.5)

### створення міні агнетів для розвязування Гауса

In [325]:
def make_agents():
    global AGENTS
    AGENTS = [] 
    for i in range(2,len(data)):
        AGENTS.append(Gaus_line_equation(data[i], data[i-2], data[i-1], get_coef3(i)))

### створення рівнянь

In [326]:
def make_equation():
    global EQUATION_ARR
    EQUATION_ARR = [np.asarray([0]*(5))]*4 # Система нормальних рівнянь
    for i in range(len(EQUATION_ARR)):
        for agent in AGENTS:
            EQUATION_ARR[i] = EQUATION_ARR[i] + agent.array[i]
    B = np.array([EQUATION_ARR[i][0] for i in range(len(EQUATION_ARR))])
    A = np.array([EQUATION_ARR[i][1:] for i in range(len(EQUATION_ARR))])
    coef = np.linalg.solve(A, B)
    return coef

### Передбачення

In [327]:
def predict(coef):
    real_data = data[2:]
    predictions = []
    for i in range(2,len(data)+1):
        prediction = coef[0] + coef[1]*data[i-2] + coef[2]*data[i-1] + coef[3] * get_coef3(i)
        predictions.append(prediction)

    print(f"\nПохибка: S = {mean_square_deviation(real_data, predictions)}")
    return predictions

## main

In [328]:
list_predictions = []
for i in range(1,4):
    prediction_type = i
    make_agents()
    coef = make_equation()

    print("\nКоефіцієнти:")
    print("a0 =", coef[0])
    print("a1 =", coef[1])
    print("a2 =", coef[2])
    print("a3 =", coef[3])

    predictions = predict(coef)
    print(predictions[-1])
    list_predictions.append(predictions)


Коефіцієнти:
a0 = -73.86418339166514
a1 = 6.830730297124161
a2 = 5.744144323040659
a3 = -0.4408105586906516

Похибка: S = 1.6791546708401615
23.012244737740986

Коефіцієнти:
a0 = -48.453395049911805
a1 = 5.445391022901181
a2 = 1.6549213473285025
a3 = -0.012003676704513975

Похибка: S = 0.974511584215631
24.16269891637188

Коефіцієнти:
a0 = -74.36694936508212
a1 = 3.3807740153595285
a2 = 5.825193295261083
a3 = -0.014032508610356112

Похибка: S = 1.9642588116621387
18.318464932399255


### neural network

In [329]:
x = []
y = []
for i in range(2,len(data)):
    pred1,pred2,pred3 = [*zip(*list_predictions)][:-1][i-2]
    x.append([data[i-2], data[i-1], pred1, pred2, pred3])
    y.append(data[i])
x = np.asarray(x)
y = np.asarray(y)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=1, activation=tf.keras.activations.relu, input_shape=(5,),
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=11) ))
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001), loss=tf.keras.losses.mean_squared_error, metrics=[tf.keras.metrics.mean_squared_error])

model.fit(x, y, epochs=700, verbose=False)
pred1,pred2,pred3 = [*zip(*list_predictions)][-1]
prediction = model.predict(np.array([[ data[-2], data[-1], pred1, pred2, pred3]]))
print(f"Прогноз: {prediction[0][0]}")

1/1 [==============================] - 0s 38ms/step
Прогноз: 26.787097930908203


In [330]:
predictions = []
for i in range(2,len(data)+1):
    pred1,pred2,pred3 = [*zip(*list_predictions)][i-2]
    prediction = model.predict(np.array([[ data[i-2], data[i-1], pred1, pred2, pred3]]))
    print(prediction)
    predictions.append(prediction[0][0])
list_predictions.append(predictions)

1/1 [==============================] - 0s 19ms/step


[[12.500978]]
1/1 [==============================] - 0s 23ms/step
[[20.117556]]
1/1 [==============================] - 0s 20ms/step
[[14.230274]]
1/1 [==============================] - 0s 20ms/step
[[19.139559]]
1/1 [==============================] - 0s 21ms/step
[[14.113716]]
1/1 [==============================] - 0s 25ms/step
[[11.181465]]
1/1 [==============================] - 0s 22ms/step
[[26.787098]]


### plot

In [331]:
import plotly.graph_objects  as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1,1+len(data))), y=data, mode='lines', name='Правдиві дані'))
i=0
for predictions in list_predictions:
    fig.add_trace(go.Scatter(x=list(range(3,3+len(predictions))), y=predictions, mode='lines', name='Передбачення'+str(i+1)))
    i+=1
fig.show()